In [40]:
#imports
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks

In [41]:
DEV = False
argvs = sys.argv
argc = len(argvs)

if argc > 1 and (argvs[1] == "--development" or argvs[1] == "-d"):
  DEV = True

if DEV:
  epochs = 12
else:
  epochs = 50

train_data_path = './data/train'
validation_data_path = './data/validation'

"""
Parameters
"""
img_width, img_height = 150, 150
batch_size = 64
samples_per_epoch = 4000
validation_steps = 1000
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 8
lr = 0.0004


In [42]:
model = Sequential()
model.add(Convolution2D(nb_filters1, conv1_size, conv1_size, border_mode ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(nb_filters2, conv2_size, conv2_size, border_mode ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), dim_ordering='th'))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=lr),
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

"""
Tensorboard log
"""
log_dir = './tf-log/'
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)
cbks = [tb_cb]

model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=cbks,
    validation_steps=validation_steps)

target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./models/model.h5')
model.save_weights('./models/weights.h5')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(150, 150,..., padding="same")`
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), padding="same")`
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  


Found 6631 images belonging to 8 classes.
Found 6631 images belonging to 8 classes.


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, validation_data=<keras.pre..., callbacks=[<keras.ca..., validation_steps=1000, steps_per_epoch=62)`


Epoch 1/50
62/62 [==============================] - 2431s 39s/step - loss: 2.6205 - acc: 0.3024 - val_loss: 1.6821 - val_acc: 0.3856
Epoch 2/50
62/62 [==============================] - 2605s 42s/step - loss: 1.6586 - acc: 0.3760 - val_loss: 1.4661 - val_acc: 0.4691
Epoch 3/50
62/62 [==============================] - 2620s 42s/step - loss: 1.5902 - acc: 0.4312 - val_loss: 1.3745 - val_acc: 0.5184
Epoch 4/50
62/62 [==============================] - 2618s 42s/step - loss: 1.5020 - acc: 0.4627 - val_loss: 1.3683 - val_acc: 0.5207
Epoch 5/50
62/62 [==============================] - 2609s 42s/step - loss: 1.4932 - acc: 0.4749 - val_loss: 1.3057 - val_acc: 0.5312
Epoch 6/50
62/62 [==============================] - 2631s 42s/step - loss: 1.4011 - acc: 0.5055 - val_loss: 1.2742 - val_acc: 0.5393
Epoch 7/50
61/62 [============================>.] - ETA: 6s - loss: 1.3924 - acc: 0.5136 

KeyboardInterrupt: 

In [ ]:
#predict_multiclass
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model

img_width, img_height = 150, 150
model_path = './models/model.h5'
model_weights_path = './models/weights.h5'
model = load_model(model_path)
model.load_weights(model_weights_path)

def predict(file):
  x = load_img(file, target_size=(img_width,img_height))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = model.predict(x)
  result = array[0]
  answer = np.argmax(result)
  if answer == 0:
    print("Label: Pizza")
  elif answer == 1:
    print("Labels: Poodle")
  elif answer == 2:
    print("Label: Rose")

  return answer

pizza_t = 0
pizza_f = 0
poodle_t = 0
poodle_f = 0
rose_t = 0
rose_f = 0

for i, ret in enumerate(os.walk('./test-data/pizza')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label: Pizza")
    result = predict(ret[0] + '/' + filename)
    if result == 0:
      pizza_t += 1
    else:
      pizza_f += 1

for i, ret in enumerate(os.walk('./test-data/poodle')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label: Poodle")
    result = predict(ret[0] + '/' + filename)
    if result == 1:
      poodle_t += 1
    else:
      poodle_f += 1

for i, ret in enumerate(os.walk('./test-data/rose')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label: Rose")
    result = predict(ret[0] + '/' + filename)
    if result == 2:
      rose_t += 1
    else:
      rose_f += 1

"""
Check metrics
"""
print("True Pizza: ", pizza_t)
print("False Pizza: ", pizza_f)
print("True Poodle: ", poodle_t)
print("False Poodle: ", poodle_f)
print("True Rose: ", rose_t)
print("False Rose: ", rose_f)